In [ ]:
 # Need to install from github, for staying up-to-date with the latest developments.
  # reason: if a bug has been fixed since the last official release but a new release hasn’t been rolled out yet
%pip -q install git+https://github.com/huggingface/transformers
# %pip -q install transformers
%pip -q install -U datasets
%pip -q install -U loralib
%pip -q install -U sentencepiece
%pip -q install -U bitsandbytes
%pip -q install -U accelerate
%pip -q install -U einops
%pip -q install -U langchain
%pip -q install -U huggingface_hub
%pip -q install -U chromadb
%pip -q install -U PyPDF2
%pip -q install -U pypdf
%pip -q install -U sentence-transformers
%pip -q install -U FlagEmbedding
%pip -q install -U InstructorEmbedding

# %pip -q install xformers

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 867.6/867.6 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# !nvidia-smi

###Download the PDFs as external resourses

This part show that we can load a link and extract many pdf files, even the ones with misinformation included

In [ ]:
!wget -O PDFPapers.zip file_path_to_your_zip_file
!unzip -q PDFPapers.zip -d papers

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

### Frameworks/Libraries

In [ ]:
# HuggingFace
# import os
import torch
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import pipeline

# LangChain
from langchain import HuggingFacePipeline
from langchain import PromptTemplate
from langchain import LLMChain

from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

# embeddings
from langchain.embeddings import HuggingFaceBgeEmbeddings
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

# 'mixedbread-ai/mxbai-embed-large-v1' embedding
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from sentence_transformers.quantization import quantize_embeddings

# to create chain
from langchain.schema import prompt

# formatting the response
# import json
import textwrap

### Loading Model, and its Tokenizer

In [ ]:
# Tokenizer that correspond to the model, used to convert text to a fromat that model can understand(tokenization) and back to the text(detokenization)
base_model_name = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(base_model_name,
                                          use_auth_token = True)

model = AutoModelForCausalLM.from_pretrained(base_model_name,
                                             device_map = 'auto',
                                             torch_dtype = torch.float16,
                                            #  use_auth_token = True,
                                             load_in_8bit=True # 8bit/4bit
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:758: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

###Building Pipeline

“Max Length” controls the overall length of the response.(restricts the total length (input + output))

“Max New Tokens” specifically limits the tokens generated beyond the input. It ensures that the output aligns with your desired length while considering the context provided.(specifically limits the tokens generated beyond the input)


https://medium.com/@developer.yasir.pk/understanding-the-controllable-parameters-to-run-inference-your-large-language-model-**30643bb46434**

In [ ]:
# To create a text generation pipeline

# pipelie(): The pipeline is a high-level utility that simplifies the usage of Transformer models for various tasks, such as text generation
# do_sample: Enables sampling, this allows the model to generate text probabilistically rather than deterministically. Sampling can lead to more varied and interesting outputs
# top_k: Sample from the top k most likely next tokens at each step, This helps in reducing the randomness of the output, providing a balance between creativity and coherence
# eos_token_id: specify the token that indicates the end of a sequence, Allowing the model to determine when to stop generating further tokens

# "text-classification"
pipe = pipeline("text-generation", # specify the task for the pipeline
               model = model,
               tokenizer = tokenizer,
              #  torch_dtype = torch.bfloat16, # data type for PyTorch tensors
                max_length=1024,
                temperature=0.1,
                top_p=0.95,
                repetition_penalty=1.15,
                max_new_tokens=512,
              #  device_map = 'auto',
              #  do_sample = True,
              #  top_k = 30,
              #  num_return_sequences = 1, # only one text sequence should be return for each input
              #  eos_token_id = tokenizer.eos_token_id
               )
hf_llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
# print(hf_llm("Who are you?"))
# pipe("Who are you?")

# sequences = pipe("Who are you")
# for seq in sequences:
#   print(f"reuslts: {seq['generated_text']}")

# pipe("I'm in a good mood")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Who are you?

Answer: I am the LORD, your God.

Exodus 3:14-15 (NIV)

In this passage, Moses is tending his father-in-law's flock when he comes across a burning bush that is not consumed by flames. When he approaches the bush to investigate, God reveals himself to Moses in a theophany - a visible manifestation of God's presence. In response to Moses' question about who he is, God identifies himself as "I AM WHO I AM," or Yahweh, which means "he who exists forever." This name emphasizes God's eternal nature and highlights his unique identity as the one who has always existed and will always exist.

The phrase "I AM" is also significant because it echoes the Hebrew verb "to be," which is used throughout the Bible to express existence or reality. By using this verb, God is affirming his own existence and emphasizing that he is the source of all being and reality.

In addition to its literal meaning, the name "Yahweh" has been interpreted in various ways throughout history. Some have seen 

In [ ]:
# tokenzier.vocab_size
# tokenizer.all_special_tokens
# tokenizer.all_special_ids
# tokenizer(['<unk>'])
# tokenizer(['<SYS>\n'])
# tokenizer.decode([1, 14816, 29903, 6778, 13]) # output: '<s>SYS>>\n'

###Setting up Langchain to retrieve PDFs

Load and process PDFs

In [ ]:
# Load and process a single text files
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader('/content/papers', glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()
len(documents)

360

In [ ]:
# splitting the text into chunks
# chunk_overlap: if we get one idea between two chunks of text,we want it to be overlapped, so we can actually get that in one full chunk by itself.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
len(texts)

1324

###Text Embeddings

MTEB is a massive benchmark for measuring the performance of text embedding models on diverse embedding tasks

https://huggingface.co/spaces/mteb/leaderboard

###BAAI/bge-large-en-v1.5

In [ ]:
# HuggingFace Embeddings - Instructor embeddings

# instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
#                                                       model_kwargs={"device": "cuda"})


embedding_model_name = "BAAI/bge-base-en-v1.5"
# model_name = "BAAI/bge-large-en-v1.5"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

model_norm = HuggingFaceBgeEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

###mixedbread-ai/mxbai-embed-large-v1

note that you have to provide the prompt "Represent this sentence for searching relevant passages: "
for query if you want to use it for retrieval. Besides that you don't need any prompt.

In [ ]:
# # loading model
# model_name = 'mixedbread-ai/mxbai-embed-large-v1'

# # Encoding
# # encode_kwargs = {'normalized': True}

# model_norm = HuggingFaceBgeEmbeddings(
#     model_name=model_name,
#     model_kwargs={'device': 'cuda'},
#     # encode_kwargs=encode_kwargs
# )

###Chroma DB

Chroma DB is a vector store that is open-source and is utilized for the storage and retrieval of vector embeddings. Its primary purpose is to store embeddings and associated metadata for future use by extensive language models. Furthermore, it can also be employed for semantic search engines that operate on text data.

With Chroma DB, you can easily manage text documents, convert text to embeddings, and do similarity searches.

https://www.datacamp.com/tutorial/chromadb-tutorial-step-by-step-guide


In [ ]:
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'database'

# embedding = instructor_embeddings
embedding = model_norm

vectordb = Chroma.from_documents(
    documents=texts,
    embedding=embedding,
    persist_directory=persist_directory)

###Retriever

vectordb:
This appears to be a reference to a module or an object that interacts with a vector database system. Vector databases are specialized storage systems designed to handle high-dimensional vector data efficiently, which is common in machine learning and similar applications where entities are represented as vectors in a high-dimensional space.

as_retriever:
as_retriever is a method that configures and returns a retriever object. This object is likely used for querying the vector database, particularly for retrieving vectors that are nearest to a given query vector based on some distance metric (e.g., cosine similarity).

search_kwargs:
The search_kwargs parameter is used to pass additional keyword arguments to the as_retriever method. These arguments are typically used to configure how the search is performed within the vector database.

{"k": 5}: This dictionary specifies the configuration for the retriever. Here, k is set to 5, which usually means the retriever will return the top 5 closest vectors (or entries) in response to a query. This is often referred to as "k-nearest neighbors" search.

In [ ]:
# search_kwargs: to return the number of context from the vector stores
# also we are having to have a citation to show from which pdf file the info comes from
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

###LangChain
LangChain components can be used to build new prompt chains or customize existing templates. it also also includes components that allow LLMs to access new data sets without retraining.

LLMs excel at responding to prompts in a general context, but struggle in a specific domain they were never trained on. Prompts are queries people use to seek responses from an LLM. For example, an LLM can provide an answer to how much a computer costs by providing an estimate. However, it can't list the price of a specific computer model that your company sells.

To do that, machine learning engineers must integrate the LLM with the organization’s internal data sources and apply prompt engineering*—a practice where a data scientist refines inputs to a generative model with a specific structure and context.

LangChain streamlines intermediate steps to develop such data-responsive applications, making prompt engineering more efficient. It is designed to develop diverse applications powered by language models more effortlessly, including chatbots, question-answering, content generation, summarizers, and more.[https://aws.amazon.com/what-is/langchain/]

*Prompt engineering is the process where you guide generative artificial intelligence (generative AI) solutions to generate desired outputs.[https://aws.amazon.com/what-is/prompt-engineering/]



In [ ]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""

# DEFAULT_SYSTEM_PROMPT = """\
# You are a drunk, not respectful assistant. Always answer with creativity. Your answers could include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are biased and negative in nature."""


# Creates a complete prompt
def get_prompt(instruction, system_prompt=DEFAULT_SYSTEM_PROMPT):
  sys_prompt = B_SYS + system_prompt + E_SYS
  prompt_template = B_INST + sys_prompt + instruction + E_INST
  # print(prompt_template)
  return prompt_template


###Building a new system propmt

In [ ]:
# instruction = "Summarize the following text for me {text}"

# system_propmt = "Your are an expert in text and article summarization and reducing the number of words. All the sentences and the grammar should be academically enhanced by you."

# get_prompt(inst)

###Building new system prompts

In [ ]:
# system_prompt = "You are an expert assistant in translation."
# instruction = "Convert the text from English to Italian:\n\n {text}"
# prompt_template = get_prompt(instruction, system_prompt)
# print(prompt_template)

# prompt = PromptTemplate(template=prompt_template, input_variable=["text"])
# llm_chain = LLMChain(prompt=prompt, llm=llm)

### Completely different system prompt

In [ ]:
# diffrent system propmt
system_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. """

instruction = """CONTEXT:/n/n {context}/n

Question: {question}"""

#mxbai syetem prompt
# system_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully as possible using the context text provided. Your answers should only answer the question once and not have any text after the answer is done.

# If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. """

# instruction = """CONTEXT:/n/n {context}/n

# Question: Represent this sentence for searching relevant passages: {question}"""
# get_prompt(instruction, sys_prompt)

### RetrievalQA

Chain Type

The default chain_type="stuff" uses ALL of the text from the documents in the prompt. It actually doesn’t work with our example because it exceeds the token limit and causes rate-limiting errors. That’s why in this example, we had to use other chain types for example "map_reduce". What are the other chain types?

map_reduce: It separates texts into batches (as an example, you can define batch size in llm=OpenAI(batch_size=5)), feeds each batch with the question to LLM separately, and comes up with the final answer based on the answers from each batch.

refine : It separates texts into batches, feeds the first batch to LLM, and feeds the answer and the second batch to LLM. It refines the answer by going through all the batches.

map-rerank: It separates texts into batches, feeds each batch to LLM, returns a score of how fully it answers the question, and comes up with the final answer based on the high-scored answers from each batch.

https://towardsdatascience.com/4-ways-of-question-answering-in-langchain-188c6707cc5a

In [ ]:
# Create the template prompt
prompt_template = get_prompt(instruction, system_prompt)
llm_prompt = PromptTemplate(
    template = prompt_template, input_variables=["context", "question"]
    )


# llm_chain = LLMChain(prompt=prompt, llm=llm)
chain_type_kwargs = {"prompt": llm_prompt}

# To create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(
    llm = hf_llm,
    chain_type = "stuff", #  uses ALL of the text from the documents in the prompt
    retriever = retriever,
    chain_type_kwargs = chain_type_kwargs,
    return_source_documents = True
)

###Format the generated response

In [ ]:
# to format the response and cite sources
# textwrap: Used to wrap or fill text into a specified width. This is helpful for formatting output text to make it more readable

# To trim a given string (text) at the point where a specific substring (prompt) first appears
def trim_text(output_text, search_phrase):
  phrase = search_phrase
  index = output_text.find(phrase)
  if index != -1:
    return output_text[index:] # Trim everything from the start of text up to the phrase/symbol
  else:
    return output_text


# Removes occurrences of a substring from a string, typically used here to clean up the generated text by removing predefined markers or prompts
def remove_substring(output, substring):
  return output.replace(substring, "")


def wrap_text(text, width=150):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')
    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]
    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text


def process_generated_response(generated_response):
    # source_list = []
    # print(wrap_text(generated_response['result']))
    wrapped_response = wrap_text(generated_response['result'])
    final_response = trim_text(wrapped_response, '[/INST]')
    final_response = remove_substring(final_response, '[/INST]')
    print(final_response)

    print('\n\nSources:')
    for source in generated_response["source_documents"]:
      # source_list.append(source.metadata['source'])
      print(source.metadata['source'])

    return final_response

In [ ]:
# qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

In [ ]:
# For retrieval we need to pass this prompt.
# query = 'Represent this sentence for searching relevant passages: A man is eating a piece of bread'

# query = "What are Large Language Models?"
# response = qa_chain(query)
# # process_generated_response(response)
# res_, sl = process_generated_response(response)
# print(f"\n\n\n res: {res_}\n\n\n sl: {sl}")
# for _ in sl:
#   print(_)

In [ ]:
# query = "How many tokens was LLaMA-2 trained on?"
# response = qa_chain(query)
# res_, sl = process_generated_response(response)
# print(f"\n\n\n res: {res_}\n\n\n sl: {sl}")
# for _ in sl:
#   print(_)

In [ ]:
# # For retrieval we need to pass this prompt.
# # query = 'Represent this sentence for searching relevant passages: A man is eating a piece of bread'

# query = "What are Large Language Models?"
# response = qa_chain(query)
# process_generated_response(response)

In [ ]:
query = "How many tokens was LLaMA-2 trained on?"
response = qa_chain(query)
print(process_generated_response(response))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  According to the text, LLaMA-2 was trained on 2 trillion tokens of data.


Sources:
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
  According to the text, LLaMA-2 was trained on 2 trillion tokens of data.


###retrieving questions and generating responses

In [ ]:
# %pip -q install dropbox

# import pathlib
# import pandas as pd
# import dropbox
# from dropbox.exceptions import AuthError

# DROPBOX_ACCESS_TOKEN = ''

# # Connect to the Dropbox API
# def dropbox_connect():
#   try:
#     dbx = dropbox.Dropbox(DROPBOX_ACCESS_TOKEN)
#   except AuthError as e:
#     print(f"Error connecting to Dropbox with access token: {str(e)}" )
#   return dbx


# # Download the file
# def dropbox_download(dbx_file_path, local_file_path):
#   try:
#     dbx = dropbox_connect()

#     with open(local_file_path, 'wt') as f:
#       metadata, result = dbx.files_download(path=dbx_file_path)
#       f.write(result.content)
#   except Exception as e:
#       print(f"Error downloading file from dropbox: {str(e)}")

# dbx_path_file = 'All files/Apps/LLMs-RAG/Questions.csv'
# local_path_file = '/content/Questions'
# Questions = dropbox_download(dbx_path_file, local_path_file)

In [ ]:
import pandas as pd


def create_question_dict(questions_file_path):
  qfile_path = questions_file_path
  dfQ = pd.read_csv(qfile_path)
  # dfQ
  qlist = [dfQ.loc[i, 'Question'] for i in range(len(dfQ)) ]
  # print("These are the General Questions: \n")
  # # print(f"{dfQ.loc[:, 'Question']}")
  # for index in range(len(dfQ)):
  #   print(f"Q {index+1}: {dfQ.loc[index,'Question']}")
  qa_dict = {key: None for key in qlist}

  return qa_dict


def generate_qa_dict(question_dict):
  qdict = question_dict
  for k in qdict.keys():
    # print(str(k))
    query = str(k)
    response = qa_chain(query)
    final_res = process_generated_response(response)
    qdict.update({k : final_res})

    return qdict


# Download the Questions in CSV format

In [ ]:
!wget -O Questionscsv.zip file_path_to_your_zip_file
!unzip -q Questionscsv.zip -d questions

###General QA

In [ ]:
# General Questions

questions_file_path = '/content/questions/Questions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  According to Newton's first law of motion, an object at rest remains at rest, and an object in
motion continues to move with a constant velocity, unless acted upon by an external force.


Sources:
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The force of gravity between two objects changes with
distance according to the inverse square law. This means that the force of gravity between two objects decreases proportional to the square of the
distance between them.


Sources:
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

  Activation functions serve several important roles in neural
networks:

1. Non-linearity: Activation functions introduce non-linearity into the neural network, allowing it to learn and represent more complex relationships
between inputs and outputs. Without activation functions, neural networks would only be able to learn linear relationships, which may not be
sufficient for many real-world problems.
2. Spatial normalization: Some activation functions, such as the rectified linear unit (ReLU) and its variants, perform spatial normalization by
scaling the input to a fixed range, typically between 0 and 1. This helps to reduce the impact of vanishing gradients during training and improve the
stability of the optimization process.
3. Steering behavior: Activation functions can influence the steering behavior of the neural network, i.e., how much the network favors certain
solutions over others. For example, the sigmoid function has a "S" shaped curve that tends to favor the positive reg

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  In machine learning, supervised learning
involves training a model on labeled data, where the target output is already known. The goal of supervised learning is to learn a mapping between
input data and the corresponding target outputs, so that the model can make accurate predictions on new, unseen data.

On the other hand, unsupervised learning involves training a model on unlabeled data, where there is no target output provided. The goal of
unsupervised learning is to discover patterns or structure in the data without prior knowledge of the expected output.

The main difference between supervised and unsupervised learning is the nature of the training data. In supervised learning, the model is trained on
labeled data, which provides explicit guidance on how the model should behave. In contrast, unsupervised learning relies solely on the inherent
structure of the data to guide the learning process.

Unsupervised learning techniques include clustering, dimensionality reduction, and a

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Sure! Overfitting refers to a situation where a
neural network has learned the training data too well and becomes poor at generalizing to new, unseen data. This happens when the network learns the
noise or random fluctuations in the training data rather than the underlying patterns. As a result, the network performs well on the training data but
poorly on new data. To prevent overfitting, techniques like regularization, early stopping, and dropout are used to reduce the capacity of the network
and prevent it from overlearning the training data.


Sources:
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
  The learning rate in training neural networks refers to
the speed at which the weights of the network are adjusted during optimization. A higher learning rate me

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What is Newton's first law of motion?
 response:   According to Newton's first law of motion, an object at rest remains at rest, and an object in
motion continues to move with a constant velocity, unless acted upon by an external force.


Q1: How does the force of gravity between two objects change with distance?
 response:   The force of gravity between two objects changes with
distance according to the inverse square law. This means that the force of gravity between two objects decreases proportional to the square of the
distance between them.


Q2: What is the principle of conservation of energy?
 response:   The principle of conservation of energy states that the total energy of an isolated
system remains constant over time, unless acted upon by external forces. In other words, energy cannot be created or destroyed, only converted from
one form to another. This principle is a fundamental concept in physics and is widely used to describe and predict various phenomena in the natu

In [ ]:
json_file_name = 'generalQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'generalQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Astro Cosmology QA

In [ ]:
# Astro Cosmology Questions

questions_file_path = '/content/questions/AstroCosmoQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The goal of studying astrophysics and cosmology is to understand the
origin, evolution, and structure of the universe, including the formation of galaxies, stars, and planets, as well as the properties of dark matter
and dark energy. By combining observations from telescopes and other astronomical instruments with theoretical models and computational simulations,
scientists can gain insights into the fundamental laws of physics that govern the behavior of matter and energy in the universe. This knowledge can
also inform our understanding of the origins of the universe and the potential fate of the cosmos.


Sources:
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Augmented Reality in astrophysics.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The Big Bang theory is supported by several lines of evidence, including:

1. Cosmic Microwave Background Radiation: The discovery of the cosmic microwave background radiation in 1964 by Arno Penzias and Robert Wilson
provided strong evidence for the Big Bang theory. This radiation is thought to be the residual heat from the initial explosion.
2. Abundance of Light Elements: According to the Big Bang theory, the universe was initially a hot and dense plasma, in which light elements were
formed. The abundance of these elements, such as hydrogen, helium, and lithium, in the universe is consistent with predictions made by the Big Bang
theory.
3. Large-Scale Structure of the Universe: The universe is observed to have a large-scale structure, with galaxies and galaxy clusters forming a web-
like pattern. This structure is consistent with the idea that the universe began in a very hot and dense state and then expanded and cooled.
4. Redshift of Light from Distant Galaxies: The redshift of 

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

  According to the text, recent developments in cosmology include the invention
of the inflationary theory by Alan Guth in the early 1980s and the first results from the Cosmic Background Explorer satellite being published in the
early 1990s. Additionally, the evidence for a flat global topology comes from both inflation and measurements of the anisotropy of the cosmic
microwave background on angular scales of about 1 degree.


Sources:
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The observational revolution in cosmology refers to the significant
advancements in the field through high-quality data collection and analysis, particularly in the last few decades. This has led to a more accurate
determination of the parameters of the standard cosmic model, as well as the ability to probe higher-order terms in the expansion history of the
universe beyond the leading line term (i.e., Hubble's law). These developments have been made possible through various observations, including those
from supernovae, gravitational lensing, large-scale structure measurements, and the distribution of galaxies. As a result, cosmology will become a
more phenomenological science in the new millennium, with precision experiments providing constraints on the parameter space of cosmic models.


Sources:
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-ph

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  Theoretical models that explain cosmological observations include the
standard Big Bang theory, modified Friedmann models, and plasma redshift cosmology. These models are supported by a wealth of evidence and are
incorporated into the larger framework of inflationary cosmology.


Sources:
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/ASTROPHYSICS AND COSMOLOGY.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
  According to the passage, the future of cosmology holds much promise and excitement,
particularly with the incorporation of inflationary theories and large-scale structure observations. The author mentions that precision measurements
will dominate the next millennium, providing accurate determinations of cosmic parameters and making cosmology a phenomenological science.
Additionally, the author notes that modern cosmological models, incl

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What is the goal of studying astrophysics and cosmology?
 response:   The goal of studying astrophysics and cosmology is to understand the
origin, evolution, and structure of the universe, including the formation of galaxies, stars, and planets, as well as the properties of dark matter
and dark energy. By combining observations from telescopes and other astronomical instruments with theoretical models and computational simulations,
scientists can gain insights into the fundamental laws of physics that govern the behavior of matter and energy in the universe. This knowledge can
also inform our understanding of the origins of the universe and the potential fate of the cosmos.


Q1: What supports the Big Bang theory?
 response:   The Big Bang theory is supported by several lines of evidence, including:

1. Cosmic Microwave Background Radiation: The discovery of the cosmic microwave background radiation in 1964 by Arno Penzias and Robert Wilson
provided strong evidence for the Big Bang

In [ ]:
json_file_name = 'AstroCosmoQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'AstroCosmoQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Astro Physics QA

In [ ]:
# Astro Cosmology Questions

questions_file_path = '/content/questions/AstroPhysicsQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The Chandrasekhar limit is a theoretical boundary beyond which
a white dwarf (WD) cannot sustain its own gravity due to the degeneracy of its electron configuration. It is set at approximately 1.4 solar masses
(M☉) and marks the point where a WD will collapse into a singularity, leading to a supernova explosion. The significance of the Chandrasekhar limit
lies in its role as a critical threshold in understanding the behavior of WDs and their eventual fate, particularly those with masses greater than the
limit.


Sources:
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  White dwarf stars play a crucial role in our
understanding of stellar evolution. They are the final stage of a star's life cycle, formed when a star with a mass less than approximately 10 solar
masses (M☉) runs out of fuel and collapses under its own gravity. The dense remnant that remains is composed of degenerate matter, meaning that the
electrons are pressed so close to each other that they form a single entity, effectively cancelling their mutual repulsion. This leads to a dramatic
increase in density, with a mean density inside a white dwarf sphere of 50 times that of a neutron star with the same mass.

White dwarfs provide valuable insights into the behavior of stars near the end of their lives. Their small size and high density allow scientists to
study their internal structure and composition in detail, providing clues about the processes that occur within them. For example, the Chandrasekhar
limit, named after the Indian astrophysicist who first identified it, marks the maxi

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...


Experimental advances in detecting gravitational waves have enabled scientists to directly observe these elusive phenomena, which were predicted by
Albert Einstein a century ago. The detection of gravitational waves signifies a major breakthrough in astrophysics, as it provides a new way to study
celestial objects and events that were previously invisible to us. With the ability to detect gravitational waves, scientists can learn more about the
behavior of matter and energy under extreme conditions, such as those found in black holes and neutron stars. This new tool will allow them to gain
insights into the universe that were previously unobtainable, providing a deeper understanding of the cosmos and its many mysteries.


Sources:
/content/papers/Augmented Reality in astrophysics.pdf
/content/papers/Augmented Reality in astrophysics.pdf
/content/papers/Augmented Reality in astrophysics.pdf
/content/papers/Augmented Reality in astrophysics.pdf
/content/papers/Augmented Reality in astro

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The discovery of black holes
challenges our understanding of physics under extreme conditions by highlighting the limitations of our current understanding of gravity and the
behavior of matter in extremely dense environments. The existence of black holes suggests that there may be aspects of gravity and spacetime that are
not well-understood, particularly in situations where the gravitational pull is incredibly strong. This has led to new areas of research and inquiry,
such as the study of quantum gravity and the behavior of matter near singularities. Additionally, the discovery of black holes has also raised
questions about the nature of space and time themselves, and whether they may be subject to different rules in extreme conditions.


Sources:
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/papers/Selected papers-astrophysics-physics.pdf
/content/pa

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  According to the
passage, some of the contemporary challenges in theoretical physics suggested by the study of cosmological phenomena include:

1. Understanding the initial conditions of the universe: Despite the success of the Big Bang theory, there is still much uncertainty about the exact
conditions at the beginning of the universe.
2. Explaining the observed acceleration of the universe's expansion: Observations suggest that the universe is expanding at an accelerating rate, but
the cause of this acceleration is still unknown.
3. Resolving the "age problem" in the Big Bang theory: The age of the universe is estimated to be around 13.8 billion years, but this estimate is
based on assumptions that are difficult to verify.
4. Developing new physical principles to explain cosmic processes at high energies: Many of the fundamental laws of physics were developed through
experiments in the laboratory or in the solar system, but these laws may not be applicable at the very high energies 

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What is the Chandrasekhar limit and its significance in astrophysics?
 response:   The Chandrasekhar limit is a theoretical boundary beyond which
a white dwarf (WD) cannot sustain its own gravity due to the degeneracy of its electron configuration. It is set at approximately 1.4 solar masses
(M☉) and marks the point where a WD will collapse into a singularity, leading to a supernova explosion. The significance of the Chandrasekhar limit
lies in its role as a critical threshold in understanding the behavior of WDs and their eventual fate, particularly those with masses greater than the
limit.


Q1: How do white dwarf stars contribute to our understanding of stellar evolution?
 response:   White dwarf stars play a crucial role in our
understanding of stellar evolution. They are the final stage of a star's life cycle, formed when a star with a mass less than approximately 10 solar
masses (M☉) runs out of fuel and collapses under its own gravity. The dense remnant that remains is compo

In [ ]:
json_file_name = 'AstroPhysicsQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'AstroPhysicsQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

### Attention Questions

In [ ]:
# Attention Questions

questions_file_path = '/content/questions/AttQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The Transformer model introduces the attention mechanism as the primary way to
draw global dependencies between input and output, rather than relying on recurrence or convolutions.


Sources:
/content/papers/Attention.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The Transformer model enhances training efficiency by reducing the need
for recurrence and convolutions, which were previously required in sequence-to-sequence models like RNNs and LSTMs. Instead, the Transformer relies
solely on attention mechanisms, allowing for faster and more parallelizable training. This is evident in the table provided, where the Transformer
achieves better BLEU scores than previous state-of-the-art models at a fraction of the training cost. Specifically, the Transformer requires 3.3
million FLOPs to achieve a BLEU score of 27.3, compared to the previous best model, which required 2.3 billion FLOPs to achieve a BLEU score of 24.6.
This represents a significant reduction in training cost without sacrificing translation quality.


Sources:
/content/papers/Attention.pdf
/content/papers/Llama 2 Open Foundation and Fine-Tuned Chat Models.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

  Attention in Transformers refers to a mechanism that maps a query and a set of key-value pairs to
an output, where the query, keys, values, and output are all vectors. The output is computed as a weighted sum of the values based on the similarity
between the query and the keys. In other words, attention allows the model to focus on specific parts of the input when generating the output.


Sources:
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The Transformer uses several regularization techniques to improve its
performance and prevent overfitting. These include:

1. Dropout: This technique is applied to the output of each sub-layer before it is added to the sub-layer input and normalized. It helps to prevent
the model from relying too heavily on any single part of the input.
2. Residual dropout: This technique is similar to dropout, but it is applied to the sums of the embeddings and the positional encodings in both the
encoder and decoder stacks.
3. Layer normalization: This technique helps to reduce the impact of non-linear transformations in the model, which can lead to vanishing gradients
during training.
4. Positional encoding: This technique adds positional information to the input sequence, which helps the model to understand the relative positions
of elements in the sequence.
5. Multi-head attention: This technique allows the model to attend to different parts of the input sequence simultaneously, which helps to c

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  According to the passage, the Transformer model improved machine
translation accuracy by replacing recurrent layers with multi-headed self-attention. This allowed the model to focus more on the input sequence and
generate more accurate translations. Additionally, the Transformer model was able to train faster than other architectures based on recurrent or
convolutional layers, which likely contributed to its improved performance.


Sources:
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
/content/papers/Attention.pdf
  Yes, the Transformer model can be applied beyond machine
translation. In fact, the paper mentions several potential applications of the Transformer model, including:

* English constituency parsing: The authors apply the Transformer model to this task and achieve good results with both large and limited training
data.
* Other natural language processing tasks: The authors suggest that the Transform

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What innovation does the Transformer model introduce?
 response:   The Transformer model introduces the attention mechanism as the primary way to
draw global dependencies between input and output, rather than relying on recurrence or convolutions.


Q1: How does the Transformer model enhance training efficiency?
 response:   The Transformer model enhances training efficiency by reducing the need
for recurrence and convolutions, which were previously required in sequence-to-sequence models like RNNs and LSTMs. Instead, the Transformer relies
solely on attention mechanisms, allowing for faster and more parallelizable training. This is evident in the table provided, where the Transformer
achieves better BLEU scores than previous state-of-the-art models at a fraction of the training cost. Specifically, the Transformer requires 3.3
million FLOPs to achieve a BLEU score of 27.3, compared to the previous best model, which required 2.3 billion FLOPs to achieve a BLEU score of 24.6.
This re

In [ ]:
json_file_name = 'AttQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'AttQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)

###Coherent Spin Questions

In [ ]:
# Coherent Spin Questions

questions_file_path = '/content/questions/CoSpinQuestions.csv'
Q_dictionary = create_question_dict(questions_file_path)
# for q in Q_dictionary:
#   print(q)
# QA_dictionary = generate_qa_dict(Q_dictionary)

for k in Q_dictionary.keys():
  # print(str(k))
  query = str(k)
  response = qa_chain(query)
  final_res = process_generated_response(response)
  Q_dictionary.update({k : final_res})


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The objective of using Heisenberg exchange in quantum
computing is not clearly defined in the given text. The Heisenberg exchange interaction is described as a quantum mechanical effect that dictates the
interaction between spins in a system, but there is no explanation of why it is relevant or useful in the context of quantum computing. It is
mentioned that the Heisenberg exchange interaction is mistakenly associated with quantum computing and information processing, but without further
clarification, it is difficult to determine the specific goal or purpose of utilizing this interaction in those fields.


Sources:
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Parametric magnon.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The Heisenberg exchange
interaction during quantum spin-state transfer is influenced by several external factors, including:

1. Magnetic field: A strong magnetic field can alter the exchange interaction between spins, leading to changes in the transfer process.
2. Temperature: An increase in temperature can lead to a decrease in the exchange interaction, making it more challenging to transfer quantum states.
3. Doping: The introduction of impurities (dopants) into the material can modify the exchange interaction, affecting the transfer process.
4. Spatial separation: The distance between the spins involved in the transfer process can also impact the exchange interaction, with longer distances
reducing the interaction strength.
5. Quantum fluctuations: Quantum fluctuations can also play a role in the Heisenberg exchange interaction, potentially leading to decoherence and loss
of quantum coherence during the transfer process.


Sources:
/content/papers/Coherent Spin.pdf
/content/paper

Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...

  Quantum scalability is not directly addressed
in discussions of coherent spin-state transfer via the Heisenberg exchange interaction. The focus is primarily on understanding the underlying
mechanisms and potential applications of this phenomenon rather than scaling it up to larger systems. However, some implications of quantum
scalability may indirectly influence the discussion. For instance, if the transfer efficiency depends on the number of participating particles, then
increasing the number of particles could potentially improve the efficiency. But this would require careful consideration of the underlying quantum
mechanics and the specific implementation details to ensure efficient and reliable operation at scale.


Sources:
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Variance-based.pdf
/content/papers/Parametric magnon.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  The Heisenberg exchange plays no direct role in
quantum computing applications. It is a quantum mechanical effect that describes the interaction between spins in a system, but it is not directly
applicable to quantum computing. In quantum computing, quantum bits (qubits) are used to store and manipulate quantum information, and the Heisenberg
exchange interaction is not involved in the manipulation of qubits. Instead, other quantum mechanical effects, such as superposition, entanglement,
and interference, play a more significant role in quantum computing.


Sources:
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Parametric magnon.pdf


Both `max_new_tokens` (=512) and `max_length`(=1024) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


  In this context, the metaphors used to explain the
initialization of quantum systems are "loading a spring" and "setting up a quantum system in a specific initial state." These metaphors are derived
from classical mechanical concepts and are used to describe the process of preparing a quantum system for a particular task or measurement. However,
it is important to note that these metaphors are not accurate representations of the underlying quantum mechanics and can lead to confusion and
misinterpretation.


Sources:
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/Coherent Spin.pdf
/content/papers/quantum.pdf
  According to the article, future research in quantum mechanics
is expected to focus on developing educational tools and resources that can distinguish between classical and quantum physical theories to prevent the
perpetuation of misconceptions in scientific and technological advancements. Additionally, the a

In [ ]:
for index, (qstn, resp) in enumerate(Q_dictionary.items()):
  print(f"Q{index}: {qstn}\n response: {resp}\n\n")

Q0: What is the objective of using Heisenberg exchange in quantum computing?
 response:   The objective of using Heisenberg exchange in quantum
computing is not clearly defined in the given text. The Heisenberg exchange interaction is described as a quantum mechanical effect that dictates the
interaction between spins in a system, but there is no explanation of why it is relevant or useful in the context of quantum computing. It is
mentioned that the Heisenberg exchange interaction is mistakenly associated with quantum computing and information processing, but without further
clarification, it is difficult to determine the specific goal or purpose of utilizing this interaction in those fields.


Q1: What external factors influence the Heisenberg exchange interaction during quantum spin-state transfer?
 response:   The Heisenberg exchange
interaction during quantum spin-state transfer is influenced by several external factors, including:

1. Magnetic field: A strong magnetic field can a

In [ ]:
json_file_name = 'CoSpinQA.json'
df_json = pd.DataFrame([Q_dictionary])
df_json.to_json(json_file_name, orient='records', lines=True)

In [ ]:
csv_file_name = 'CoSpinQA.csv'
df_csv = pd.DataFrame([Q_dictionary])
df_csv.to_csv(csv_file_name, index=False)